In [112]:
# load libraries
import datasets
import re

In [93]:
# load video data
video_data = datasets.Dataset.load_from_disk("../data/videos/")
descriptions = video_data['description']
titles = video_data['title']

In [64]:
# clean descriptions
des = descriptions[7567]
des

" Former President Donald Trump weighs in on allegations against the Biden family's business dealings on 'Hannity.'\\n\\nSubscribe to Fox News! https://bit.ly/2vBUvAS\\nWatch more Fox News Video: http://vi..."

In [102]:
# clean all descriptions
pre_processed_descriptions = []
pre_processed_titles = []
for d in descriptions:
    d = re.sub(
        '(Subscribe to)|'
        '((http).*(Watch))|'
        '((http).*(\.\.\.))|'
        '(News Agency)|(News)|'
        '(Fox)|(CNN)|(BBC)|(DW)|(CCTV)|(Al Jazeera)|(#foxnews)|(#fox)|(#ingrahamangle)'
        '(\\\\u0026 Friends Weekend)|'
        '(\\\\n)|(\\\\r)', '', d)
    pre_processed_descriptions.append(d)
for t in titles:
    t = re.sub(
        '(Subscribe to)|'
        '((http).*(Watch))|'  # complete youtube links
        '((http).*(\.\.\.))|'  # cut off links
        '(Newsfeed)|(News Agency)|(News)|'
        '(Fox)|(CNN)|(BBC)|(DW)|(CCTV)|(Al Jazeera)|(#foxnews)|(#fox)|(#ingrahamangle)'  # cha
        '(\\\\u0026 Friends Weekend)|'
        '(\\\\n)|(\\\\r)|'
        '(\| The Stream)', '', t)
    pre_processed_titles.append(t)

AttributeError: 'dict' object has no attribute 'append'

In [101]:
# TODO: assign values or create new dataset
# Process the dataset - add a column with the length of the context texts
dataset_with_length = video_data.map(lambda x: {"length": len(x["title"])})
# add values to new column
tokenized_dataset = video_data.map(lambda x: pre_processed_titles, batched=True)

Map:   0%|          | 0/7908 [00:00<?, ? examples/s]

Map:   0%|          | 0/7908 [00:00<?, ? examples/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'list'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.